In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
import dash_table as dt
from dash.dash_table.Format import Group
from dash.dependencies import Input, Output
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
#from animal_shelter import AnimalShelter


from aacCRUD import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

usr = 'aacuser'
pw = 'yh0okRBJkaFVCJnw'

str = "mongodb+srv://" + usr + ":" + pw + "@cluster0.cyqsq." + "mongodb.net/?retryWrites=true&w=majority"
#shelter = CRUD(str, 'AAC', 'animals')
shelter = CRUD('mongodb://localhost:27017', 'animals', 'shelterA')

cursor = shelter.readAll()
df = pd.DataFrame.from_records((cursor))
print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Willi Blanco CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='submit-button-one',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='submit-button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='submit-button-three',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='reset')          
        ]


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
         page_size=100,
        style_table={'height':'300px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(230,230,230)',
            'fontWeight':'bold'        
        },
        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },
        
        #tooltips that we are going to use on the table so that we know what information we are looking at
        tooltip ={i: {
        'value': i,
        'use_with': 'both'  # both refers to header & data cell
    } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        
        #sorting features that we are going to use
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])



#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data')],
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two','n_clicks'), 
               Input('submit-button-three','n_clicks'),Input('submit-button-four','n_clicks'),
               Input('filter-type', 'value')])
                               
def update_dashboard(bt1,bt2,bt3,bt4):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
                        
    if (int(bt1) >= 1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                   {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        bt2, bt3, bt4 = 0

    elif (int(bt2)>= 1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
                   {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        bt1, bt3 ,bt4 = 0

    elif (int(bt3)>=1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                   {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
        bt1, bt2, bt4 = 0

    elif(int(bt4)>=1):
        df = pd.Dataframe.from_records(shelter.read())
        bt1, bt2, bt3 = 0

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')


    return data

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values=values, names=names, title='Percentage of breeds available')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
                               
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and pop up
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
        dl.Tooltip(dff['breed']),        
        dl.Popup([
        html.H1("Animal Name"),
        html.P(dff.loc[0,'name'])
            ])
                ])
                   ])]


app

                         _id animal      breed  age   health neutered  \
0   62edadc4fea3e6dd0ea439c5    cat  shorthair    2     good    false   
1   62edae57b0c3d39e9accc911    cat  shorthair    2     good    false   
2   62edae6dab89df527584c0dc    cat  shorthair    2     good    false   
3   62edaf031ef279ee99920bd7    cat  shorthair    2     good    false   
4   62edaf26c807a57782b8b904    cat  shorthair    2     good    false   
5   62edaf26c807a57782b8b905    cat       tabi    1     good    false   
6   62edaf26c807a57782b8b906    cat  longthair    3    great    false   
7   62edaf26c807a57782b8b907    cat  shorthair    2  average     true   
8   62edaf26c807a57782b8b908    dog     french    5      bad     true   
9   62edaf26c807a57782b8b909    cat     sphynx    6  average     true   
10  62edaf26c807a57782b8b90a    cat       tabi    7  average     true   
11  62edaf26c807a57782b8b90b    dog     buldog    4     good    false   
12  62edaf26c807a57782b8b90c    cat  shorthair    4